<a href="https://colab.research.google.com/github/WendyValdes/Azure-reconhecimento-audio/blob/main/Azure_reconhecer_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalação da Biblioteca Azure Speech SDK

Esta célula instala a biblioteca `azure-cognitiveservices-speech`, que é essencial para usar os serviços de Text-to-Speech (TTS) e Speech-to-Text (STT) do Azure. A execução deste comando garante que todas as dependências necessárias estejam disponíveis no ambiente do Colab.

In [20]:
!pip install azure-cognitiveservices-speech

## Teste de Text-to-Speech (TTS)

Esta célula demonstra como usar o serviço de Text-to-Speech do Azure. Ele configura a chave de assinatura e a região do serviço, define a voz desejada (FranciscaNeural em português do Brasil) e então converte um texto fornecido em áudio, que é reproduzido. Ao final, uma mensagem confirma que o áudio foi gerado com sucesso.

In [21]:
import azure.cognitiveservices.speech as speechsdk

# Configuração do serviço de fala do Azure
speech_key = "suachave"
service_region = "brazilsouth"  # Exemplo: "brazilsouth", defina a região do seu serviço

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_synthesis_voice_name = "pt-BR-FranciscaNeural"  # Define a voz em português do Brasil

# Inicializa o sintetizador de fala
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

# Texto que será transformado em áudio
texto = "Olá! Esse é um teste do Azure Text-to-Speech."

# Executa o processo de TTS de forma assíncrona e aguarda a conclusão
speech_synthesizer.speak_text_async(texto).get()
print("Áudio gerado com sucesso!")

Áudio gerado com sucesso!


## Carregamento de Arquivo de Áudio

Esta célula permite carregar um arquivo de áudio do seu computador local para o ambiente do Google Colab. É fundamental para fornecer arquivos para o serviço de Speech-to-Text. Após a execução, o nome do arquivo carregado será armazenado na variável `uploaded_filename` para uso posterior.

In [22]:
from google.colab import files

# Permite ao usuário fazer upload de arquivos
uploaded = files.upload()

# Itera sobre os arquivos carregados e imprime seus nomes e tamanhos
for fn in uploaded.keys():
  print(f'Arquivo enviado pelo usuário: "{fn}" com tamanho {len(uploaded[fn])} bytes')
  uploaded_filename = fn # Armazena o nome do arquivo carregado para uso futuro

Saving PTT-20260119-WA0004.opus to PTT-20260119-WA0004 (1).opus
Arquivo enviado pelo usuário: "PTT-20260119-WA0004 (1).opus" com tamanho 3181 bytes


## Conversão de Áudio (OPUS para WAV)

Esta célula é responsável por converter um arquivo de áudio no formato `.opus` (comumente usado por aplicativos de mensagens) para o formato `.wav`, que é o formato preferencial para o serviço de Speech-to-Text do Azure. Ele instala as ferramentas necessárias (ffmpeg) e, em seguida, utiliza `subprocess` para chamar diretamente o `ffmpeg` para realizar a conversão, garantindo que o arquivo de áudio esteja no formato correto para o reconhecimento.

In [23]:
# Instalar pydub (ainda útil para outras coisas) e ffmpeg para conversão de áudio
!pip install pydub
!apt-get update && apt-get install -y ffmpeg

import os
import subprocess # Importa subprocess para chamadas diretas ao ffmpeg

# Verifica se o arquivo foi carregado e se é um arquivo .opus
if 'uploaded_filename' in globals() and uploaded_filename.endswith('.opus'):
    original_opus_file = uploaded_filename
    # Sanitiza o nome do arquivo para o comando ffmpeg, evitando problemas com caracteres especiais
    simple_opus_file = "uploaded_audio.opus"
    wav_file = os.path.splitext(simple_opus_file)[0] + '.wav'

    # Renomeia o arquivo original para um nome mais simples se ele existir
    if os.path.exists(original_opus_file):
        os.rename(original_opus_file, simple_opus_file)
        print(f"Arquivo renomeado de '{original_opus_file}' para '{simple_opus_file}'")
    else:
        print(f"Erro: O arquivo original '{original_opus_file}' não foi encontrado.")
        simple_opus_file = None

    # Se o arquivo .opus estiver disponível, procede com a conversão
    if simple_opus_file:
        opus_file_to_convert = simple_opus_file
        print(f"Convertendo {opus_file_to_convert} para {wav_file} usando ffmpeg...")

        try:
            # Usa subprocess para chamar ffmpeg diretamente
            # -i para entrada, -c:a pcm_s16le para WAV não comprimido, -ar 16000 para taxa de amostragem,
            # -ac 1 para mono, -y para sobrescrever o arquivo de saída sem perguntar
            command = ["ffmpeg", "-i", opus_file_to_convert, "-c:a", "pcm_s16le", "-ar", "16000", "-ac", "1", "-y", wav_file]
            result = subprocess.run(command, capture_output=True, text=True, check=True)
            print("Conversão concluída com sucesso!")
            print("FFmpeg stdout:", result.stdout)
            print("FFmpeg stderr:", result.stderr) # ffmpeg frequentemente imprime o progresso em stderr
            converted_filename = wav_file
        except subprocess.CalledProcessError as e:
            print(f"Erro ao converter o arquivo com ffmpeg: {e}")
            print("FFmpeg stdout:", e.stdout)
            print("FFmpeg stderr:", e.stderr)
            converted_filename = None
        except Exception as e:
            print(f"Erro inesperado durante a conversão: {e}")
            converted_filename = None
    else:
        print("Não foi possível processar o arquivo .opus para conversão.")
        converted_filename = None
else:
    print("Nenhum arquivo .opus carregado ou o nome do arquivo não foi encontrado.")
    converted_filename = None

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed

## Reconhecimento de Fala (Speech-to-Text - STT)

Esta célula utiliza o serviço de Speech-to-Text do Azure para transcrever áudio de um arquivo `.wav` para texto. Ela configura a chave de assinatura e a região do serviço, e crucialmente, define o idioma de reconhecimento para português do Brasil (`pt-BR`). Ele então usa o nome do arquivo `.wav` convertido para o reconhecimento. O resultado da transcrição é impresso, ou uma mensagem de erro é exibida caso o reconhecimento falhe.

In [24]:
import azure.cognitiveservices.speech as speechsdk

speech_key = "suaxhave"
service_region = "brazilsouth"

speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_config.speech_recognition_language = "pt-BR" # Define o idioma de reconhecimento para português do Brasil

# Usa o nome do arquivo convertido, se disponível
if 'converted_filename' in globals() and converted_filename:
    audio_config = speechsdk.AudioConfig(filename=converted_filename)
    print(f"Usando o arquivo de áudio convertido: {converted_filename}")
elif 'uploaded_filename' in globals() and uploaded_filename.endswith('.wav'):
    # Caso um arquivo .wav tenha sido carregado diretamente e não precise de conversão
    audio_config = speechsdk.AudioConfig(filename=uploaded_filename)
    print(f"Usando o arquivo de áudio carregado: {uploaded_filename}")
else:
    print("Erro: Nenhum arquivo de áudio WAV válido disponível para reconhecimento.")
    audio_config = None # Garante que audio_config seja None para evitar erros futuros


if audio_config:
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Reconhecendo fala do arquivo de áudio...")

    result = speech_recognizer.recognize_once_async().get()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Texto reconhecido: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("Não foi possível reconhecer a fala.")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Reconhecimento cancelado: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Detalhes do erro: {}".format(cancellation_details.error_details))
else:
    print("Não foi possível configurar o Speech Recognizer devido à falta de arquivo de áudio válido.")

Usando o arquivo de áudio convertido: uploaded_audio.wav
Reconhecendo fala do arquivo de áudio...
Texto reconhecido: Bom dia, tudo bem?
